In [2]:
import csv
import os
import sys

os.environ["SPARK_HOME"]="/Users/batalabdulaev/BigData/spark/spark"
os.environ["PYSPARK_PYTHON"]="/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8"
os.environ["PYSPARK_DRIVER_PYTHON"]="/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

In [3]:
import pyspark
from pyspark.sql import SparkSession, Row

import pyspark.sql.functions as F
from pyspark.sql.functions import round
from pyspark.sql.types import (
    StructType, 
    StructField, 
    StringType, 
    IntegerType, 
    DoubleType
)

In [4]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geopandas.tools import sjoin
from pyspark.sql.functions import unix_timestamp
import folium
from folium.plugins import HeatMap, HeatMapWithTime

In [46]:
from IPython.display import IFrame
def embed_map(m):
    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

In [6]:
packages = "graphframes:graphframes:0.8.1-spark2.4-s_2.11"

In [7]:
conf = (
    pyspark.SparkConf()
        .set("spark.executor.memory", "1g")
        .set("spark.executor.core", "2")
        .set("spark.driver.host", "127.0.0.1")  # if you have some troubles with address binding
        .setMaster("local[2]")
)

In [8]:
spark = SparkSession \
    .builder \
    .config(conf=conf) \
    .getOrCreate()

In [9]:
sc = spark.sparkContext
sc

<SparkContext master=local[2] appName=pyspark-shell>

1 ЗАДАНИЕ

определите для каждой станции количество начала поездок и количество завершения поездок
сопоставьте станции с кварталами города (zones) и определите суммы количества начала и завершения для каждого квартала
выведите по убыванию количества поездок и
отобразите в виде картограмм (Choropleth).

In [10]:
BASE_PATH = "/Users/batalabdulaev/BigData/datasets"
QUARTER_NYC = "/Users/batalabdulaev/BigData/datasets/NYC Taxi Zones.geojson"

In [11]:
trips_data_path ="file://{}/201902-citibike-tripdata.csv".format(BASE_PATH)

In [12]:
df_trips = spark.read.load(trips_data_path, 
                           format="csv", 
                           header="true", 
                           inferSchema="true",
                           sep=",")

print("Total number of trips:", df_trips.count())
df_trips.show(2)

Total number of trips: 943744
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|           starttime|            stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|         219|2019-02-01 00:00:...|2019-02-01 00:03:...|            3494|E 115 St & Lexing...|             40.797911|               -73.9423|          3501|E 118 St & Madiso...|          40.8014866| 

In [13]:
df_trips.printSchema()

root
 |-- tripduration: integer (nullable = true)
 |-- starttime: string (nullable = true)
 |-- stoptime: string (nullable = true)
 |-- start station id: string (nullable = true)
 |-- start station name: string (nullable = true)
 |-- start station latitude: double (nullable = true)
 |-- start station longitude: double (nullable = true)
 |-- end station id: string (nullable = true)
 |-- end station name: string (nullable = true)
 |-- end station latitude: double (nullable = true)
 |-- end station longitude: double (nullable = true)
 |-- bikeid: integer (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth year: integer (nullable = true)
 |-- gender: integer (nullable = true)



In [23]:
df_trips.schema

StructType(List(StructField(tripduration,IntegerType,true),StructField(starttime,StringType,true),StructField(stoptime,StringType,true),StructField(start station id,StringType,true),StructField(start station name,StringType,true),StructField(start station latitude,DoubleType,true),StructField(start station longitude,DoubleType,true),StructField(end station id,StringType,true),StructField(end station name,StringType,true),StructField(end station latitude,DoubleType,true),StructField(end station longitude,DoubleType,true),StructField(bikeid,IntegerType,true),StructField(usertype,StringType,true),StructField(birth year,IntegerType,true),StructField(gender,IntegerType,true)))

In [24]:
df_trips.columns

['tripduration',
 'starttime',
 'stoptime',
 'start station id',
 'start station name',
 'start station latitude',
 'start station longitude',
 'end station id',
 'end station name',
 'end station latitude',
 'end station longitude',
 'bikeid',
 'usertype',
 'birth year',
 'gender']

In [25]:
df_trips_started = df_trips.groupBy("start station id")\
    .agg(F.count("start station id")\
    .alias("Trips started"))\
    .withColumnRenamed("start station id", "s_id")
df_trips_started.show()

+----+-------------+
|s_id|Trips started|
+----+-------------+
| 296|         1795|
|3606|          215|
| 467|         1276|
|3414|          656|
|3368|          745|
|3517|          299|
|3441|          263|
|3526|          604|
| 447|         2512|
|3312|          788|
|3121|          291|
|3249|          341|
|3167|         2820|
| 307|         3028|
|3491|          398|
|3057|          480|
| 334|         2905|
|3553|          798|
|3241|          453|
|3299|          256|
+----+-------------+
only showing top 20 rows



In [26]:
df_trips_ended = df_trips.groupBy("end station id")\
    .agg(F.count("end station id")\
    .alias("Trips ended"))\
    .withColumnRenamed("end station id", "s_id")
df_trips_ended.show()

+----+-----------+
|s_id|Trips ended|
+----+-----------+
| 467|       1312|
| 296|       1825|
|3414|        681|
|3606|        224|
|3441|        268|
|3368|        769|
|3517|        301|
|3526|        606|
| 447|       2499|
|3312|        791|
|3249|        340|
|3121|        295|
|3167|       2693|
|3491|        399|
| 307|       3006|
|3057|        451|
| 334|       2912|
|3241|        432|
|3299|        262|
|3553|        818|
+----+-----------+
only showing top 20 rows



In [27]:
df_trips_all = df_trips_started.join(df_trips_ended, on="s_id", how="outer")
df_trips_all.show()

+----+-------------+-----------+
|s_id|Trips started|Trips ended|
+----+-------------+-----------+
| 119|          171|        217|
| 120|          593|        524|
| 127|         2940|       3010|
| 128|         2639|       2671|
| 143|         1387|       1380|
| 144|          441|        513|
| 146|         1656|       1673|
| 150|         1753|       1664|
| 151|         3771|       3802|
| 157|         1129|       1163|
| 161|         2602|       2617|
| 164|         1862|       1860|
| 167|         2663|       2658|
| 168|         3738|       3741|
| 173|         3701|       3732|
| 174|         2445|       2413|
| 195|         2340|       2289|
|2000|         1344|       1537|
|2002|         1783|       1866|
|2003|         3182|       3174|
+----+-------------+-----------+
only showing top 20 rows



In [28]:
df_trips_started = df_trips_started.withColumnRenamed("s_id","start station id")
df_trips_ended = df_trips_ended.withColumnRenamed("s_id","end station id")
df_trips_started.printSchema()

root
 |-- start station id: string (nullable = true)
 |-- Trips started: long (nullable = false)



2 zadanie

In [29]:
with open(QUARTER_NYC) as file:
    zones_geojson = json.load(file)
zones_geojson

In [32]:
column_name_list = [i for i, _ in zones_geojson["features"][0]["properties"].items()] + ["geometry"]
column_name_list

['shape_area',
 'objectid',
 'shape_leng',
 'location_id',
 'zone',
 'borough',
 'geometry']

In [35]:
def make_rows_zone(data):
    
    for item in data:
        arr_row = []
        for key, value in item["properties"].items():
            arr_row.append(value) 
            
        polygons = []
        for polygon in item["geometry"]["coordinates"]:
            polygons.append(Polygon(polygon[0]))
            
        arr_row.append(MultiPolygon(polygons=polygons))
        
        yield arr_row

In [36]:
df_zones_pn = pd.DataFrame(make_rows_zone(zones_geojson["features"]), columns=column_name_list)
df_zones_pn.head(5)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/dtypes/inference.py:420: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  iter(obj)  # Can iterate over it.
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/dtypes/inference.py:421: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  len(obj)  # Has a length associated with it.
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/io/formats/printing.py:115: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituen

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,(POLYGON ((-74.18445299999996 40.6949959999999...
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,(POLYGON ((-73.82337597260663 40.6389870471767...
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,(POLYGON ((-73.84792614099985 40.8713422339999...
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,(POLYGON ((-73.97177410965318 40.7258212813370...
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,(POLYGON ((-74.17421738099989 40.5625680859999...


dataframe with zones

In [38]:
gdf_zones = gpd.GeoDataFrame(df_zones_pn, geometry=df_zones_pn["geometry"])
gdf_zones.head()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/geopandas/_vectorized.py:142: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/geopandas/_vectorized.py:142: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  aout[:] = out


,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."
5,0.000606460984581,6,0.150490542523,6,Arrochar/Fort Wadsworth,Staten Island,"MULTIPOLYGON (((-74.06367 40.60220, -74.06351 ..."
6,0.000389787989274,7,0.107417171123,7,Astoria,Queens,"MULTIPOLYGON (((-73.90414 40.76752, -73.90325 ..."
7,0.000026587716279,8,0.0275906911574,8,Astoria Park,Queens,"MULTIPOLYGON (((-73.92334 40.77513, -73.92398 ..."
8,0.000338443803197,9,0.0997840924705,9,Auburndale,Queens,"MULTIPOLYGON (((-73.78502 40.76104, -73.78486 ..."
9,0.0000607235737749,24,0.0469999619287,24,Bloomingdale,Manhattan,"MULTIPOLYGON (((-73.95954 40.79872, -73.96004 ..."


In [39]:
df_stations_begin = df_trips.select(F.col("start station id").alias("s_id"),
                                  F.col("start station name").alias("station_name"),
                              F.col("start station latitude").alias("lat"), 
                              F.col("start station longitude").alias("lng"))\
                            .distinct()

+----+--------------------+------------------+------------------+
|s_id|        station_name|               lat|               lng|
+----+--------------------+------------------+------------------+
|3170|W 84 St & Columbu...|       40.78499979|      -73.97283406|
|3236|  W 42 St & Dyer Ave| 40.75898481399634|-73.99379968643188|
|3232| Bond St & Fulton St| 40.68962188790333|-73.98304268717766|
| 437|Macon St & Nostra...|     40.6809833854|    -73.9500479759|
| 517|Pershing Square S...|         40.751581|         -73.97791|
|3256|Pier 40 - Hudson ...|  40.7277140777778|-74.01129573583603|
|3063|Nostrand Ave & My...|       40.69527008|      -73.95238108|
|3660|     W 16 St & 8 Ave|40.741021509002664|  -74.001384973526|
|3686|Gansevoort St & H...|40.739448201304036|-74.00507032871246|
| 341|Stanton St & Mang...|       40.71782143|      -73.97628939|
| 223|     W 13 St & 7 Ave|       40.73781509|      -73.99994661|
|3503|Madison Ave & E 1...|          40.80295|        -73.942954|
|3431|    

In [42]:
df_stations_end = df_trips.select(F.col("end station id").alias("s_id"),
                                  F.col("end station name").alias("station_name"),
                              F.col("end station latitude").alias("lat"), 
                              F.col("end station longitude").alias("lng"))\
                            .distinct()

df_stations = df_stations_begin.union(df_stations_end).distinct().filter(F.col("s_id")!="NULL")

df_stations.show(10)

+----+--------------------+------------------+------------------+
|s_id|        station_name|               lat|               lng|
+----+--------------------+------------------+------------------+
|3170|W 84 St & Columbu...|       40.78499979|      -73.97283406|
|3236|  W 42 St & Dyer Ave| 40.75898481399634|-73.99379968643188|
|3232| Bond St & Fulton St| 40.68962188790333|-73.98304268717766|
| 437|Macon St & Nostra...|     40.6809833854|    -73.9500479759|
| 517|Pershing Square S...|         40.751581|         -73.97791|
|3256|Pier 40 - Hudson ...|  40.7277140777778|-74.01129573583603|
|3063|Nostrand Ave & My...|       40.69527008|      -73.95238108|
|3660|     W 16 St & 8 Ave|40.741021509002664|  -74.001384973526|
|3686|Gansevoort St & H...|40.739448201304036|-74.00507032871246|
| 341|Stanton St & Mang...|       40.71782143|      -73.97628939|
+----+--------------------+------------------+------------------+
only showing top 10 rows



In [40]:
# bc_zones = spark.sparkContext.broadcast(gdf_zones)
# # bc_zones

In [142]:
df_stations_pn = df_stations.toPandas()
df_stations_pn

,s_id,station_name,lat,lng
0,3170,W 84 St & Columbus Ave,40.785000,-73.972834
1,3236,W 42 St & Dyer Ave,40.758985,-73.993800
2,3232,Bond St & Fulton St,40.689622,-73.983043
3,437,Macon St & Nostrand Ave,40.680983,-73.950048
4,517,Pershing Square South,40.751581,-73.977910
...,...,...,...,...
765,3338,2 Ave & E 99 St,40.786259,-73.945526
766,3105,N 15 St & Wythe Ave,40.724055,-73.955736
767,3639,Harborside,40.719252,-74.034234
768,3214,Essex Light Rail,40.712774,-74.036486


In [143]:
points = gpd.GeoDataFrame(df_stations_pn,
                          geometry=gpd.points_from_xy(df_stations_pn["lng"], 
                                                      df_stations_pn["lat"]))
points.head()

,s_id,station_name,lat,lng,geometry
0,3170,W 84 St & Columbus Ave,40.785000,-73.972834,POINT (-73.97283 40.78500)
1,3236,W 42 St & Dyer Ave,40.758985,-73.993800,POINT (-73.99380 40.75898)
2,3232,Bond St & Fulton St,40.689622,-73.983043,POINT (-73.98304 40.68962)
3,437,Macon St & Nostrand Ave,40.680983,-73.950048,POINT (-73.95005 40.68098)
4,517,Pershing Square South,40.751581,-73.977910,POINT (-73.97791 40.75158)


In [144]:
sz = sjoin(points, gdf_zones, how="left")
sz.head()

,s_id,station_name,lat,lng,geometry,index_right,shape_area,objectid,shape_leng,location_id,zone,borough
0,3170,W 84 St & Columbus Ave,40.785000,-73.972834,POINT (-73.97283 40.78500),231.0,0.000204715440774,239,0.0636261152958,239,Upper West Side South,Manhattan
1,3236,W 42 St & Dyer Ave,40.758985,-73.993800,POINT (-73.99380 40.75898),47.0,0.0000942538425377,48,0.0437467441431,48,Clinton East,Manhattan
2,3232,Bond St & Fulton St,40.689622,-73.983043,POINT (-73.98304 40.68962),15.0,0.000124168267356,25,0.0471458199319,25,Boerum Hill,Brooklyn
3,437,Macon St & Nostrand Ave,40.680983,-73.950048,POINT (-73.95005 40.68098),21.0,0.000322957654799,17,0.093522632948,17,Bedford,Brooklyn
4,517,Pershing Square South,40.751581,-73.977910,POINT (-73.97791 40.75158),164.0,0.0000743150675972,170,0.0457690091051,170,Murray Hill,Manhattan


In [40]:
#

In [47]:
sz_DataF = pd.DataFrame(sz)
SZsparkDF=spark.createDataFrame(sz_DataF.astype(str))

In [48]:
szJoin = df_trips_all.join(SZsparkDF, on="s_id", how="left")\
    .select("s_id","station_name","s_id","Trips started","Trips ended"\
           ,"location_id","zone")
szJoin.show(10)

+----+--------------------+----+-------------+-----------+-----------+--------------------+
|s_id|        station_name|s_id|Trips started|Trips ended|location_id|                zone|
+----+--------------------+----+-------------+-----------+-----------+--------------------+
| 146|Hudson St & Reade St| 146|         1656|       1673|        231|TriBeCa/Civic Center|
| 120|Lexington Ave & C...| 120|          593|        524|         17|             Bedford|
| 128|MacDougal St & Pr...| 128|         2639|       2671|        125|           Hudson Sq|
| 144| Nassau St & Navy St| 144|          441|        513|         66|  DUMBO/Vinegar Hill|
| 150|   E 2 St & Avenue C| 150|         1753|       1664|          4|       Alphabet City|
| 143|Clinton St & Jora...| 143|         1387|       1380|         33|    Brooklyn Heights|
| 127|Barrow St & Hudso...| 127|         2940|       3010|        158|Meatpacking/West ...|
| 157|Henry St & Atlant...| 157|         1129|       1163|         33|    Brookl

In [49]:
df_trips_with_zones = szJoin.groupBy("zone", "location_id")\
    .agg(F.sum("Trips started")\
    .alias("Total zone trips starts"),\
    F.sum("Trips ended")\
    .alias("Total zone trips ends"))
df_trips_with_zones.show()

+--------------------+-----------+-----------------------+---------------------+
|                zone|location_id|Total zone trips starts|Total zone trips ends|
+--------------------+-----------+-----------------------+---------------------+
|   East Harlem North|         74|                   7835|                 7889|
|Long Island City/...|        146|                   2614|                 2569|
|Meatpacking/West ...|        158|                  12284|                12575|
|          Greenpoint|        112|                  11892|                11870|
| Lincoln Square West|        143|                  11857|                11761|
|Central Harlem North|         42|                   1606|                 1610|
|       Midtown South|        164|                  14508|                14578|
|  World Trade Center|        261|                   4323|                 4979|
|  Brooklyn Navy Yard|         34|                   2095|                 2445|
|Williamsburg (Sou...|      

In [50]:
df_trips_with_zones.sort("Total zone trips starts", ascending=False).show()

+--------------------+-----------+-----------------------+---------------------+
|                zone|location_id|Total zone trips starts|Total zone trips ends|
+--------------------+-----------+-----------------------+---------------------+
|        East Village|         79|                  43333|                43170|
|        East Chelsea|         68|                  38967|                39395|
|         Murray Hill|        170|                  32572|                32578|
|            Union Sq|        234|                  30527|                31464|
|Greenwich Village...|        113|                  26485|                27054|
|        Clinton East|         48|                  25364|                25241|
|TriBeCa/Civic Center|        231|                  24744|                25350|
|     Lower East Side|        148|                  23070|                23236|
|            Flatiron|         90|                  22976|                23238|
|    Garment District|      

In [51]:
trips_choro = df_trips_with_zones.toPandas()
trips_choro.head()

,zone,location_id,Total zone trips starts,Total zone trips ends
0,East Harlem North,74,7835.0,7889
1,Long Island City/Queens Plaza,146,2614.0,2569
2,Meatpacking/West Village West,158,12284.0,12575
3,Greenpoint,112,11892.0,11870
4,Lincoln Square West,143,11857.0,11761


In [55]:
m1 = folium.Map()
folium.Choropleth(
    geo_data=zones_geojson,
    data=trips_choro,
    columns=["location_id", "Total zone trips starts"],
    name="Number of trip starts per zone",
    legend_name="Number of trip starts per zone",
    key_on="feature.properties.location_id",
    highlight=True,
    nan_fill_color="black",
    nan_fill_opacity=0.1,
    fill_color="YlOrRd",
    fill_opacity=0.5,
    line_opacity=0.4,
).add_to(m1)
m1.fit_bounds(m1.get_bounds())
embed_map(m1)

In [53]:
m2 = folium.Map()

folium.Choropleth(
    geo_data=zones_geojson,
    data=trips_choro,
    columns=["location_id", "Total zone trips ends"],
    name="Number of trip ends per zone",
    legend_name="Number of trips ends per zone",
    key_on="feature.properties.location_id",
    highlight=True,
    nan_fill_color="black",
    nan_fill_opacity=0.1,
    fill_color="YlOrRd",
    fill_opacity=0.8,
    line_opacity=0.4,
).add_to(m2)
m2.fit_bounds(m2.get_bounds())
embed_map(m2)

# Задание 3

In [62]:
from math import cos, asin, sqrt
def haversine_func(x1, y1, x2, y2):
    p = 0.017453292519943295
    a = 0.5 - cos((x2 - x1) * p)/2 + cos(x1 * p) * cos(x2 * p) * (1 - cos((y2 - y1) * p)) / 2
    return 12742 * asin(sqrt(a))

def distance(rows):
    for row in rows:
        if row["start station id"] != row["end station id"]:
            distance = haversine_func(row["start station latitude"],row["start station longitude"],row["end station latitude"], row["end station longitude"])*1000
            yield Row(**row.asDict(), Distance=distance)

In [63]:
df_trips_distance = df_trips.rdd.mapPartitions(distance).toDF()\
    .select("start station id", "start station name", "end station id","end station name", "distance")\
    .filter(F.col("start station id") != F.col("end station id"))
df_trips_distance.show(2)

+----------------+--------------------+--------------+--------------------+------------------+
|start station id|  start station name|end station id|    end station name|          distance|
+----------------+--------------------+--------------+--------------------+------------------+
|            3494|E 115 St & Lexing...|          3501|E 118 St & Madiso...|430.16041257402225|
|             438| St Marks Pl & 1 Ave|           236| St Marks Pl & 2 Ave|143.63994354824555|
+----------------+--------------------+--------------+--------------------+------------------+
only showing top 2 rows



In [64]:
df_trips_distance.select("start station id", "start station name","end station id", "end station name", "distance")\
.sort("distance", ascending=False).show(5)

+----------------+--------------------+--------------+--------------------+------------------+
|start station id|  start station name|end station id|    end station name|          distance|
+----------------+--------------------+--------------+--------------------+------------------+
|            3576|Park Pl & Vanderb...|          3547|Broadway & Moylan Pl|15326.431486590483|
|            3537|Carlton Ave & Dea...|          3545| Broadway & W 122 St|14598.812101590232|
|            3537|Carlton Ave & Dea...|          3545| Broadway & W 122 St|14598.812101590232|
|            3377|Carroll St & Bond St|          3536| W 116 St & Broadway|14578.585483875455|
|            3384|     Smith St & 3 St|          3552| W 113 St & Broadway|14389.392491567256|
+----------------+--------------------+--------------+--------------------+------------------+
only showing top 5 rows



In [65]:
df_trips_distance.select("start station id", "start station name",\
                         "end station id", "end station name", "distance")\
                        .sort("distance", ascending=True).show(5)

+----------------+--------------------+--------------+--------------------+-----------------+
|start station id|  start station name|end station id|    end station name|         distance|
+----------------+--------------------+--------------+--------------------+-----------------+
|             517|Pershing Square S...|           519|Pershing Square N...|36.73587142648394|
|             517|Pershing Square S...|           519|Pershing Square N...|36.73587142648394|
|             519|Pershing Square N...|           517|Pershing Square S...|36.73587142648394|
|             519|Pershing Square N...|           517|Pershing Square S...|36.73587142648394|
|             519|Pershing Square N...|           517|Pershing Square S...|36.73587142648394|
|             517|Pershing Square S...|           519|Pershing Square N...|36.73587142648394|
|             519|Pershing Square N...|           517|Pershing Square S...|36.73587142648394|
|             519|Pershing Square N...|           517|Pershi

In [66]:
# df_trips_sel = df_trips_distance.select("start station id", "start station name",\
#                          "end station id", "end station name", "distance")
df_trips_distance.select(F.max("distance")).show()

+------------------+
|     max(distance)|
+------------------+
|15326.431486590483|
+------------------+



In [67]:
df_trips_pandas = df_trips_distance.toPandas()
df_trips_pandas.head()

,start station id,start station name,end station id,end station name,distance
0,3494,E 115 St & Lexington Ave,3501,E 118 St & Madison Ave,430.160413
1,438,St Marks Pl & 1 Ave,236,St Marks Pl & 2 Ave,143.639944
2,3571,Bedford Ave & Bergen St,3549,Grand Ave & Bergen St,821.725136
3,167,E 39 St & 3 Ave,477,W 41 St & 8 Ave,1443.121686
4,3458,W 55 St & 6 Ave,3443,W 52 St & 6 Ave,231.949666


In [68]:
df_trips_distance.select(F.avg("distance")).show()

+------------------+
|     avg(distance)|
+------------------+
|1640.3749148416223|
+------------------+



In [69]:
df_trips_distance.select(F.stddev("distance")).show()

+---------------------+
|stddev_samp(distance)|
+---------------------+
|   1288.6771757096985|
+---------------------+



In [71]:
# df_trips_pandas["distance"].median()

1640.3749148415022

In [72]:
# df_trips_pandas["distance"].std()

1288.6771757097122

In [73]:
# df_trips_pandas["distance"].mean()

1262.874571344554

In [76]:
df_trips_distance.select(F.col("distance").cast('float')).approxQuantile("distance",[0.5],0)[0]

1262.87451171875

4 задание

In [112]:
df_trips.printSchema()

root
 |-- tripduration: integer (nullable = true)
 |-- starttime: string (nullable = true)
 |-- stoptime: string (nullable = true)
 |-- start station id: string (nullable = true)
 |-- start station name: string (nullable = true)
 |-- start station latitude: double (nullable = true)
 |-- start station longitude: double (nullable = true)
 |-- end station id: string (nullable = true)
 |-- end station name: string (nullable = true)
 |-- end station latitude: double (nullable = true)
 |-- end station longitude: double (nullable = true)
 |-- bikeid: integer (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth year: integer (nullable = true)
 |-- gender: integer (nullable = true)



In [113]:
df_start = df_trips.select(F.col("start station id").alias("s_id"), \
F.date_format(F.col("starttime"),"yyyy-MM-dd").alias("starttime"))\
.filter("s_id != 'NULL'").groupBy("s_id","starttime").agg(F.count("starttime").alias('count_start'))
df_start.sort("s_id").show()

+----+----------+-----------+
|s_id| starttime|count_start|
+----+----------+-----------+
| 119|2019-02-12|          4|
| 119|2019-02-13|         11|
| 119|2019-02-09|          2|
| 119|2019-02-07|          9|
| 119|2019-02-02|          5|
| 119|2019-02-08|          6|
| 119|2019-02-22|          2|
| 119|2019-02-15|          7|
| 119|2019-02-11|         15|
| 119|2019-02-04|          5|
| 119|2019-02-03|          5|
| 119|2019-02-06|         11|
| 119|2019-02-01|          6|
| 119|2019-02-05|          9|
| 119|2019-02-14|         11|
| 119|2019-02-10|          2|
| 119|2019-02-26|          7|
| 119|2019-02-25|          5|
| 119|2019-02-19|          6|
| 119|2019-02-23|          3|
+----+----------+-----------+
only showing top 20 rows



In [114]:
df_start.groupBy('s_id').agg(F.count('s_id'), F.sum('count_start'))\
.select(F.col('s_id'),round(F.col('sum(count_start)')/ F.col('count(s_id)'),2).alias('AVG'))\
.sort('s_id').show()

+----+------+
|s_id|   AVG|
+----+------+
| 119|  6.11|
| 120| 21.18|
| 127| 105.0|
| 128| 94.25|
| 143| 49.54|
| 144| 15.75|
| 146| 59.14|
| 150| 62.61|
| 151|134.68|
| 157| 40.32|
| 161| 92.93|
| 164|  66.5|
| 167| 95.11|
| 168| 133.5|
| 173|132.18|
| 174| 87.32|
| 195| 83.57|
|2000|  48.0|
|2002| 63.68|
|2003|113.64|
+----+------+
only showing top 20 rows



In [115]:
# Завершение поездок

In [116]:
df_end = df_trips.select(F.col("end station id").alias("s_id"), \
F.date_format(F.col("stoptime"),"yyyy-MM-dd").alias("stop_time"))\
.filter("s_id != 'NULL'").groupBy("s_id","stop_time").agg(F.count("stop_time").alias('count_stop_time'))
df_start.sort("s_id").show()

+----+----------+-----------+
|s_id| starttime|count_start|
+----+----------+-----------+
| 119|2019-02-12|          4|
| 119|2019-02-13|         11|
| 119|2019-02-09|          2|
| 119|2019-02-07|          9|
| 119|2019-02-02|          5|
| 119|2019-02-08|          6|
| 119|2019-02-22|          2|
| 119|2019-02-15|          7|
| 119|2019-02-11|         15|
| 119|2019-02-04|          5|
| 119|2019-02-03|          5|
| 119|2019-02-06|         11|
| 119|2019-02-01|          6|
| 119|2019-02-05|          9|
| 119|2019-02-14|         11|
| 119|2019-02-10|          2|
| 119|2019-02-26|          7|
| 119|2019-02-25|          5|
| 119|2019-02-19|          6|
| 119|2019-02-23|          3|
+----+----------+-----------+
only showing top 20 rows



In [117]:
df_end.groupBy('s_id').agg(F.count('s_id'), F.sum('count_stop_time'))\
.select(F.col('s_id'),round(F.col('sum(count_stop_time)')/ F.col('count(s_id)'),2).alias('AVG'))\
.sort('s_id').show()

+----+------+
|s_id|   AVG|
+----+------+
| 119|  7.75|
| 120| 18.71|
| 127|103.79|
| 128| 95.39|
| 143| 49.29|
| 144| 18.32|
| 146| 59.75|
| 150| 59.43|
| 151|135.79|
| 157| 41.54|
| 161| 93.46|
| 164| 66.43|
| 167| 94.93|
| 168|133.61|
| 173|128.69|
| 174| 86.18|
| 195| 81.75|
|2000| 54.89|
|2002| 66.64|
|2003|113.36|
+----+------+
only showing top 20 rows



In [118]:
_trip_end_ = df_end.toPandas()
_trip_end_.head()

,s_id,stop_time,count_stop_time
0,3315,2019-02-01,27
1,363,2019-02-01,42
2,422,2019-02-01,35
3,3087,2019-02-01,25
4,3164,2019-02-01,52


In [119]:
_trip_start_ = df_start.toPandas()
_trip_start_.head()

,s_id,starttime,count_start
0,167,2019-02-01,76
1,3315,2019-02-01,34
2,3164,2019-02-01,97
3,363,2019-02-01,41
4,389,2019-02-01,20


In [128]:
m3 = folium.Map()
folium.Choropleth(
    geo_data=zones_geojson,
    data=_trip_start_,
    columns=["s_id", "count_start"],
    name="trip start",
    key_on="feature.properties.location_id",
    highlight=True,
    nan_fill_color="black",
    nan_fill_opacity=0.1,
    fill_color="YlOrRd",
    fill_opacity=0.5,
    line_opacity=0.4,
).add_to(m3)
m3.fit_bounds(m3.get_bounds())
embed_map(m3)

In [129]:
m4 = folium.Map()
folium.Choropleth(
    geo_data=zones_geojson,
    data=_trip_end_,
    columns=["s_id", "count_stop_time"],
    name="trip end",
    key_on="feature.properties.location_id",
    highlight=True,
    nan_fill_color="black",
    nan_fill_opacity=0.1,
    fill_color="YlOrRd",
    fill_opacity=0.5,
    line_opacity=0.4,
).add_to(m4)
m4.fit_bounds(m4.get_bounds())
embed_map(m4)

In [ ]:
# sc.stop()